<a href="https://colab.research.google.com/github/gvanathip/webscraping/blob/main/taladthai_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install selenium

     |████████████████████████████████| 904 kB 3.1 MB/s 


In [2]:
pip install webdriver_manager

In [3]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InR

In [4]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
import datetime
import pandas as pd
import numpy as np

In [5]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [6]:
url = "https://talaadthai.com/product-search/result?subcat_id=10132&per_page=50&page=1"

driver.get(url)
time.sleep(10)

element = driver.find_element_by_xpath('//*[@id="__layout"]/div/div/div[2]/div[4]/div/div/div/div[2]/div[1]/h5')
last_page = int(np.ceil(int(element.get_attribute('innerHTML').split()[1])/50))

In [7]:
text = []
for i in range(last_page):
    URL = url[:-1]+str(i+1)
    driver.get(URL)
    time.sleep(10)
    element = driver.find_element_by_xpath('//*[@id="__layout"]/div/div/div[2]/div[4]/div/div/div/div[3]')
    text.append(element.get_attribute('innerText'))

In [8]:
product = []
detail = []
for i in text:
  for j in i.split('\n'):
    if j[-4:] == '2021':
      detail.append(j)
    else:
      product.append(j)

In [9]:
min_price = []
max_price = []
unit = []
discount = []
date_ = []
for i in detail:
  #print(i.split(' '))
  min_price.append(i.split(' ')[0])
  max_price.append(i.split(' ')[2])
  unit.append(i.split(' ')[3]+i.split(' ')[4]+i.split(' ')[5])
  discount.append(i.split(' ')[7])
  date_.append(i.split(' ')[-1])

In [10]:
from pandas import DataFrame
table = list(zip(product, min_price, max_price, unit, discount, date_))
df = DataFrame(table,columns=['product','min_price','max_price','unit','discount','date_'])
df.head()

,product,min_price,max_price,unit,discount,date_
0,กระเพาะหมู,65,70,฿/กิโลกรัม,0.00%,03/09/2021
1,ก๋วยเตี๋ยว เส้นเล็ก,25,27,฿/กิโลกรัม,0.00%,03/09/2021
2,ก๋วยเตี๋ยว เส้นใหญ่,20,20,฿/กิโลกรัม,0.00%,03/09/2021
3,ขนมจีน,20,20,฿/กิโลกรัม,0.00%,03/09/2021
4,ขอบกระด้ง(ขอบดำ),200,200,฿/กิโลกรัม,0.00%,03/09/2021


In [11]:
df['avg_price'] = (df['min_price'].astype(float)+df['max_price'].astype(float))/2
df.head()

,product,min_price,max_price,unit,discount,date_,avg_price
0,กระเพาะหมู,65,70,฿/กิโลกรัม,0.00%,03/09/2021,67.5
1,ก๋วยเตี๋ยว เส้นเล็ก,25,27,฿/กิโลกรัม,0.00%,03/09/2021,26.0
2,ก๋วยเตี๋ยว เส้นใหญ่,20,20,฿/กิโลกรัม,0.00%,03/09/2021,20.0
3,ขนมจีน,20,20,฿/กิโลกรัม,0.00%,03/09/2021,20.0
4,ขอบกระด้ง(ขอบดำ),200,200,฿/กิโลกรัม,0.00%,03/09/2021,200.0


In [12]:
df['source'] = 'ตลาดไท'
df.head()

,product,min_price,max_price,unit,discount,date_,avg_price,source
0,กระเพาะหมู,65,70,฿/กิโลกรัม,0.00%,03/09/2021,67.5,ตลาดไท
1,ก๋วยเตี๋ยว เส้นเล็ก,25,27,฿/กิโลกรัม,0.00%,03/09/2021,26.0,ตลาดไท
2,ก๋วยเตี๋ยว เส้นใหญ่,20,20,฿/กิโลกรัม,0.00%,03/09/2021,20.0,ตลาดไท
3,ขนมจีน,20,20,฿/กิโลกรัม,0.00%,03/09/2021,20.0,ตลาดไท
4,ขอบกระด้ง(ขอบดำ),200,200,฿/กิโลกรัม,0.00%,03/09/2021,200.0,ตลาดไท


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
pip install xlsxwriter

     |████████████████████████████████| 148 kB 4.0 MB/s 


In [17]:
#out_path = "/content/drive/MyDrive/public_database/taladthai_price.xlsx"

#writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
#df.to_excel(writer, index=False)
#writer.save()

In [21]:
exist = pd.read_excel(r'/content/drive/MyDrive/public_database/taladthai_price.xlsx','Sheet1')
exist.head()

,product,min_price,max_price,unit,discount,date_,avg_price,source
0,กระเพาะหมู,65.0,70.0,฿/กิโลกรัม,0.00%,03/09/2021,67.5,ตลาดไท
1,ก๋วยเตี๋ยว เส้นเล็ก,25.0,27.0,฿/กิโลกรัม,0.00%,03/09/2021,26.0,ตลาดไท
2,ก๋วยเตี๋ยว เส้นใหญ่,20.0,20.0,฿/กิโลกรัม,0.00%,03/09/2021,20.0,ตลาดไท
3,ขนมจีน,20.0,20.0,฿/กิโลกรัม,0.00%,03/09/2021,20.0,ตลาดไท
4,ขอบกระด้ง(ขอบดำ),200.0,200.0,฿/กิโลกรัม,0.00%,03/09/2021,200.0,ตลาดไท


In [25]:
with_new = pd.concat([exist,df])
with_new.head()

,product,min_price,max_price,unit,discount,date_,avg_price,source
0,กระเพาะหมู,65,70,฿/กิโลกรัม,0.00%,03/09/2021,67.5,ตลาดไท
1,ก๋วยเตี๋ยว เส้นเล็ก,25,27,฿/กิโลกรัม,0.00%,03/09/2021,26.0,ตลาดไท
2,ก๋วยเตี๋ยว เส้นใหญ่,20,20,฿/กิโลกรัม,0.00%,03/09/2021,20.0,ตลาดไท
3,ขนมจีน,20,20,฿/กิโลกรัม,0.00%,03/09/2021,20.0,ตลาดไท
4,ขอบกระด้ง(ขอบดำ),200,200,฿/กิโลกรัม,0.00%,03/09/2021,200.0,ตลาดไท


In [24]:
final = with_new.drop_duplicates()
final.head()

,product,min_price,max_price,unit,discount,date_,avg_price,source
0,กระเพาะหมู,65,70,฿/กิโลกรัม,0.00%,03/09/2021,67.5,ตลาดไท
1,ก๋วยเตี๋ยว เส้นเล็ก,25,27,฿/กิโลกรัม,0.00%,03/09/2021,26.0,ตลาดไท
2,ก๋วยเตี๋ยว เส้นใหญ่,20,20,฿/กิโลกรัม,0.00%,03/09/2021,20.0,ตลาดไท
3,ขนมจีน,20,20,฿/กิโลกรัม,0.00%,03/09/2021,20.0,ตลาดไท
4,ขอบกระด้ง(ขอบดำ),200,200,฿/กิโลกรัม,0.00%,03/09/2021,200.0,ตลาดไท


In [26]:
out_path = "/content/drive/MyDrive/public_database/taladthai_price.xlsx"

writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
final.to_excel(writer, index=False)
writer.save()

In [29]:
# authenticate
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
# create, and save df
from gspread_dataframe import set_with_dataframe
title = 'taladthai_price'
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, final) 
# include_index=False, include_column_header=True, resize=False